In [1]:
# !git clone https://github.com/simulamet-host/conditional-polyp-diffusion.git
!pip install omegaconf
!pip install pytorch-lightning
!pip install taming-transformers-rom1504
!pip install kornia
!pip install openai-clip

In [2]:
from omegaconf import OmegaConf
import os, sys, datetime, glob, importlib, csv
import random
import torch

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import sys, types
mpi4py = types.ModuleType("mpi4py")
class _DummyComm:
    def Get_size(self): return 1
    def Get_rank(self): return 0
    def bcast(self, x, root=0): return x
class _DummyMPI: COMM_WORLD = _DummyComm()
mpi4py.MPI = _DummyMPI()
sys.modules["mpi4py"] = mpi4py

In [3]:
#unet_config:
# params:
#         image_size: 128
#         in_channels: 6
#         out_channels: 3
#         model_channels: 128
#         attention_resolutions:
#         - 32
#         - 16
#         - 8
#         num_res_blocks: 2
#         channel_mult:
#         - 1
#         - 4
#         - 8
# data:
#  train:
#       target: ldm.data.kvasir.KvasirSegTrain
#       params:
#         size: 512
#     validation:
#       target: ldm.data.kvasir.KvasirSegEval
#       params:
#         size: 512

configs = [OmegaConf.load(cfg) for cfg in ["./configs/latent-diffusion/kvasir-ldm-vq4-.yaml"]]

In [4]:
# data_train.txt, data_eval.txt 가 존재하지 않는다면, 이 cell을 실행해서 생성해야됌
data_root = "../../../01_data/02_preproc/02_abnormal/"
data_set_name = "P2"

data_path = os.path.join(data_root, data_set_name, "images")

def create_data_csv(data_path, ratio=0.8):
    data_list = os.listdir(data_path)
    random.shuffle(data_list)
    train_list = data_list[:int(len(data_list)*ratio)]
    test_list = data_list[int(len(data_list)*ratio):]

    with open(os.path.join("../../../01_data/02_preproc/02_abnormal/P2/", "data_train.txt"), "w") as f: # path
        for item in train_list:
            f.write(item+"\n")

    with open(os.path.join("../../../01_data/02_preproc/02_abnormal/P2/", "data_eval.txt"), "w") as f: # path
        for item in test_list:
            f.write(item+"\n")

create_data_csv(data_path)

In [5]:
# latent-diffusion/ldm/data/kvasir에서 KvasirSegTrain와 KvasirSegEval 수정

# class KvasirSegTrain(SegmentationBase):
#     def __init__(self, size=None, random_crop=False, interpolation="bicubic"):
#         super().__init__(data_csv='/content/data_train.txt', # data_train.txt가 있는 위치 수정
#                          data_root='/content/drive/MyDrive/velum/images', # 데이터 있는 위치로 수정
#                          segmentation_root='/content/drive/MyDrive/velum/masks', # 데이터 있는 위치로 수정
#                          size=size, random_crop=random_crop, interpolation=interpolation,
#                          n_labels=2)

# class KvasirSegEval(SegmentationBase):
#     def __init__(self, size=None, random_crop=False, interpolation="bicubic"):
#         super().__init__(data_csv='/content/data_eval.txt', # data_eval.txt가 있는 위치로 수정
#                          data_root='/content/drive/MyDrive/velum/images', # 데이터 있는 위치로 수정
#                          segmentation_root='/content/drive/MyDrive/velum/masks', # 데이터 있는 위치로 수정
#                          size=size, random_crop=random_crop, interpolation=interpolation,
#                          n_labels=2)

In [6]:
import numpy as np
print(np.__version__)

1.26.4


In [7]:
# from conditional-polyp-diffusion/latent-diffusion/ldm/models/diffusion/ddpm.py 교체
# main.py 파일 17번째 줄 아래와 같이 수정
# from pytorch_lightning.utilities import rank_zero_only
from ldm.util import instantiate_from_config
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

model = instantiate_from_config(configs[0].model)
model = model.to("cuda")
model.logvar = model.logvar.to("cuda")
model.learning_rate = 1.0e-06

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 215.23 M params.
Keeping EMAs of 218.
cpu
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 128, 128) = 49152 dimensions.
making attention of type 'vanilla' with 512 in_channels
Restored from ../../../03_model/abnormal/P2/vqvae/VQVAE-epoch=105.ckpt with 0 missing and 55 unexpected keys
Spatial Rescaler mapping from 2 to 3 channels after resizing.


In [8]:
# data loader setup
data = instantiate_from_config(configs[0].data)
data.prepare_data()
data.setup()
print("#### Data #####")
for k in data.datasets:
    print(f"{k}, {data.datasets[k].__class__.__name__}, {len(data.datasets[k])}")

#### Data #####
train, KvasirSegTrain, 380
validation, KvasirSegEval, 96


In [9]:
import pytorch_lightning as pl
print(pl.__version__)

2.5.6


In [11]:
saving_ckpt_path = "../../../03_model/abnormal/P2/masked_ldm"

checkpoint_callback = ModelCheckpoint(
    dirpath=saving_ckpt_path,  # Directory to save checkpoints
    filename='latent_diffusion',  # Filename format
    monitor='val/loss',  # Metric to monitor
    mode='min',          # Mode for monitoring ('min' or 'max')
    save_top_k=3         # Number of top models to save
)

# trainer = Trainer(
#     accelerator="gpu",
#     devices=1,
#     strategy=None,
#     max_epochs=1000,
#     callbacks=[checkpoint_callback]
# )

trainer = Trainer(
    accelerator = "gpu" if torch.cuda.is_available() else "cpu",
    devices     = 1,              # 또는 devices=[0]
    # strategy    = None,         # None 대신 "auto"
    max_epochs  = 1000,
    callbacks   = [checkpoint_callback],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
/home/gil/anaconda3/envs/YS_pt/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [12]:
trainer.fit(model, data)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name              | Type             | Params | Mode 
---------------------------------------------------------------
0 | model             | DiffusionWrapper | 215 M  | train
1 | model_ema         | LitEma           | 0      | train
2 | first_stage_model | VQModelInterface | 55.3 M | eval 
3 | cond_stage_model  | SpatialRescaler  | 6      | train
---------------------------------------------------------------
215 M     Trainable params
55.3 M    Non-trainable params
270 M     Total params
1,082.208 Total estimated model params size (MB)
303       Modules in train mode
174       Modules in

LatentDiffusion: Also optimizing conditioner params!
Setting up LambdaLR scheduler...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/gil/anaconda3/envs/YS_pt/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:527: Found 174 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.
